In [1]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.ensemble
import lime
import lime.lime_tabular

# util
from xaibenchmark import load_adult as la

In [3]:
data = la.load_csv_data('adult', root_path='../data')

def preprocess(*data_df): 
    def process_single(df):
        
        cat_df = pd.get_dummies(df, columns=data.categorical_features.keys())
        missing_cols = {cat+'_'+str(attr) for cat in data.categorical_features \
                        for attr in data.categorical_features[cat]} - set(cat_df.columns)
        for c in missing_cols:
            cat_df[c] = 0
            
        cont_idx = list(set(data.data.keys()) - set(data.categorical_features.keys()))
        cat_idx = [cat+'_'+str(attr) for cat in data.categorical_features \
                   for attr in data.categorical_features[cat]]
        idx = cont_idx + cat_idx
        return cat_df[idx]
        
    # Preprocess function for one-hot encoding categorical data
    return [process_single(df) for df in data_df]

train, dev, test = preprocess(data.data, data.data_dev, data.data_test)
labels_train, labels_dev, labels_test = data.target, data.target_dev, data.target_test

In [4]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train.to_numpy().reshape(-1))

RandomForestClassifier(n_estimators=500)

In [5]:
y_pred = rf.predict(dev)

In [6]:
y_true = labels_dev.to_numpy().reshape(-1)

In [7]:
print('Classification report')
print('{:->60}'.format(''))
print(sklearn.metrics.classification_report(y_true, y_pred))

Classification report
------------------------------------------------------------
              precision    recall  f1-score   support

       <=50K       0.89      0.93      0.91      7400
        >50K       0.74      0.62      0.68      2368

    accuracy                           0.86      9768
   macro avg       0.81      0.78      0.79      9768
weighted avg       0.85      0.86      0.85      9768



In [8]:
import xaibenchmark as xb

In [53]:
class LimeExplainer(xb.Explainer):
    
    def __init__(self, train_data, train_labels, model, feature_names, target_names, positive_label, discretize_continuous=True):
        
        self.explainer = lime.lime_tabular.LimeTabularExplainer(train_data, feature_names=feature_names,
                                                   class_names=target_names,
                                                   discretize_continuous=discretize_continuous)
        self.train = train_data
        self.train_labels = train_labels
        self.positive_label = positive_label
        self.model = model
        self.kernel_width = np.sqrt(train_data.shape[1]) * .75
        
    def explain_instance(self, instance, predictor, num_features=10):
        self.explanation = self.explainer.explain_instance(instance, predictor, num_features=num_features)
        self.instance = instance
        return self.explanation
    
    @xb.metric
    def area(self):
        kernel_width = np.sqrt(self.train.shape[1]) * .75
        kernel_dimension = self.train.shape[1]
        return (kernel_width * np.sqrt(2*np.pi))**kernel_dimension
    
    @xb.metric
    def coverage(self):
        weighted_instances = self.get_weighted_instances()
        return sum([weight for _,weight in weighted_instances]) / len(weighted_instances)
    
    @xb.metric
    def furthest_distance(self):
        kernel_width = np.sqrt(self.train.shape[1]) * .75
        def kernel(distance):
            return np.sqrt(np.exp(-distance**2/kernel_width**2))
        training_instances = self.train.to_numpy()
        distance_instances = (self.distance(self.instance, instance) for instance in training_instances)
        weighted_distances = (distance * kernel(distance) for distance in distance_instances)
        return sum(weighted_distances)

    @xb.metric
    def accuracy(self):
        """ the fraction of the same predicted label count between ML model and the explainer """
        y_pred = self.get_model_prediction()
        y_true = self.train_labels.to_numpy().reshape(-1)
        fraction = sum(y_true == y_pred)
        return fraction / len(y_true)

    @xb.metric
    def balance(self):
        """ the proportion of positive and negative labels from the model's output """
        y_pred = self.get_model_prediction()
        pos = sum(y_pred == self.positive_label)
        neg = len(y_pred) - pos
        return pos / neg

    @xb.utility
    def distance(self, x, y):
        return np.linalg.norm(x-y)
    
    @xb.utility
    def get_weighted_instances(self):     
        if hasattr(self, 'explanation'):
            kernel_width = np.sqrt(self.train.shape[1]) * .75
            def kernel(distance):
                return np.sqrt(np.exp(-distance**2/kernel_width**2))
            return [(instance, kernel(self.distance(self.instance, instance))) for instance in self.train.to_numpy()]
        return []
    
    @xb.utility
    def get_explained_instance(self):
        return self.instance
    
    @xb.utility
    def get_training_data(self):
        return self.train

    @xb.utility
    def get_model_prediction(self):
        return self.model.predict(self.train)
    

In [54]:
exp = LimeExplainer(train, labels_train, rf, train.keys(), data.target_names, ">50K", discretize_continuous=False)

In [55]:
exp.explain_instance(test.iloc[1], rf.predict_proba, num_features=10)

In [56]:
exp.report()

{('accuracy', 0.99995612688106),
 ('area', 2.589477453233665e+139),
 ('balance', 0.3159170948559552),
 ('coverage', 1.5792597810605184e-05),
 ('furthest_distance', 5.169826688431765)}

In [57]:
exp.infer_metrics()

inferred metrics: {'coverage', 'accuracy', 'area_hc_normalised', 'balance', 'area', 'inverse_coverage', 'furthest_distance'}


In [58]:
exp.report()

{('accuracy', 0.99995612688106),
 ('area', 2.589477453233665e+139),
 ('area_hc_normalised', 19.537233873073223),
 ('balance', 0.3159170948559552),
 ('coverage', 1.5792597810605184e-05),
 ('furthest_distance', 5.169826688431765),
 ('inverse_coverage', 63320.8045941923)}

8362